# TP1 - Logiciels (iGraph)

In [ ]:
from igraph import *

## Import data

In [ ]:
g = read('FacebookAnon.gml')

## Affichage graphe

In [ ]:
plot(g)

## Réarranger la position des sommets pour avoir une meilleure vue de la structure

In [ ]:
layout = g.layout('fr')

In [ ]:
plot(g, layout=layout)

## Calculer le nombre de composantes connexes.

In [ ]:
composantes = g.clusters()

In [ ]:
print('On observe', len(composantes), 'compoantes connexes !')

Attention, on peut afficher la composante géante de la manière suivante :

In [ ]:
compoanteGeante = g.clusters().giant()

In [ ]:
plot(compoanteGeante)

In [ ]:
uneComposante = g.subgraph(g.clusters()[18])

In [ ]:
print(g.clusters()[18])

In [ ]:
layout = uneComposante.layout('fr')

In [ ]:
plot(uneComposante, layout=layout)

In [ ]:
uneAutreComposante = g.subgraph(g.clusters()[17])

In [ ]:
layout = uneAutreComposante.layout('fr')
plot(uneAutreComposante, layout=layout)

In [ ]:
composantes = g.clusters()

In [ ]:
# composantes.membership
palette = drawing.colors.RainbowPalette(len(composantes))

colors = [palette.get(i) for i in composantes.membership]

style = {
    'vertex_color': colors
}

plot(g, **style)

In [ ]:
plot(composantes)

## Calculer le degré de chaque sommet

In [ ]:
style['vertex_size'] = [max(d, 7) for d in g.degree()]
plot(g, **style)

## Afficher la distribution des degrés dans un histogramme

In [ ]:
print('Le degre max est', g.maxdegree())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.figure(figsize=(10, 6))

hist_degree = np.array([[b[1], b[2]] for b in g.degree_distribution().bins()])

plt.bar(hist_degree[:, 0], hist_degree[:, 1])
plt.show()

plt.figure(figsize=(10, 6))
plt.scatter(hist_degree[:, 0], hist_degree[:, 1])
plt.xscale('log')

In [ ]:
h = Histogram(2)

In [ ]:
h.add_many(g.degree())

In [ ]:
print(h)

## Retirer les labels

In [ ]:
g.vs['label'] = ['' for _ in g.vs()]

layout = g.layout('fr')


plot(g, layout=layout, **style)

## Détecter les communautés dans ce graphe

In [ ]:
communities = g.community_multilevel()

In [ ]:
del style['vertex_size']
del style['vertex_color']

In [ ]:
layout = g.layout('fr', niter=5000)

In [ ]:
plot(communities, mark_groups=True, margin=40, layout=layout, **style)

## Test 3

In [ ]:
layout = g.layout('fr3d')

Xn=[layout[k][0] for k in range(len(g.vs()))]# x-coordinates of nodes
Yn=[layout[k][1] for k in range(len(g.vs()))]# y-coordinates
Zn=[layout[k][2] for k in range(len(g.vs()))]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in g.es():
    Xe+=[layout[e.tuple[0]][0],layout[e.tuple[1]][0], None]# x-coordinates of edge ends
    Ye+=[layout[e.tuple[0]][1],layout[e.tuple[1]][1], None]
    Ze+=[layout[e.tuple[0]][2],layout[e.tuple[1]][2], None]
import chart_studio.plotly as py
import plotly.graph_objs as go

communities = g.community_multilevel()
colors = [i for i in communities.membership]

trace1 = go.Scatter3d(
    x=Xe,
    y=Ye,
    z=Ze,
    name='Friendship',
    mode='lines',
    line=dict(color='rgb(125,125,125)', width=1),
    hoverinfo='none'
)

trace2=go.Scatter3d(
    x=Xn,
    y=Yn,
    z=Zn,
    mode='markers',
    name='Facebook user',
    marker=dict(
        symbol='circle',
        size=12,
        color=colors,
        colorscale='Viridis',
        line=dict(color='rgb(50,50,50)', width=0.5)
    ),
    text=[v.attributes()['label'] for v in g.vs()],
    hoverinfo='text'
)

axis=dict(
    showbackground=False,
    showline=False,
    zeroline=False,
    showgrid=False,
    showticklabels=False,
    title=''
)

layout = go.Layout(
         title="Facebook",
         width=800,
         height=800,
         showlegend=True,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
        dict(
            showarrow=False,
            text="Data source: <br><a href='https://igraph.org/%27%3E[1] iGraph</a><br>"+
               "<a href='https://plotly.com/%27%3E[1] Plotly</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
                size=14
            )
        )
    ],
)
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)
fig